In [ ]:
import sys
sys.path.append("..")

from src.utils.pgsql import pg_df
import pandas as pd
import numpy as np
import json

In [ ]:
# import league specific scoring rules
rules_df = pg_df(
    "SELECT scoring_settings FROM leagues WHERE id = '1260107285024219136'"
    )

# If it's a string, parse it as JSON
if isinstance(rules_df, str):
    scoring_dict = json.loads(rules_df)
else:
    scoring_dict = rules_df

for rule, points in scoring_dict.items():
    print(f"{rule}: {points} points")

scoring_settings: 0    {'sack': 1.0, 'fgm_40_49': 4.0, 'pass_int': -2...
Name: scoring_settings, dtype: object points


In [ ]:
# analysing seasonal data looking at prior seasons correlation to the following season
weekly_df = pg_df("""
    SELECT * 
    FROM nfl_weekly_stats 
    WHERE season >= 2010
    AND season_type = 'REG'
    """)

# create seasonal aggregation
season_cols = [
'player_id', 'season', 'player_name',
'position', 'position_group', 'recent_team_old', 

       'completions', 'attempts', 
       'passing_yards','passing_tds', 'interceptions', 
       'passing_first_downs', 'passing_2pt_conversions',

       'carries', 'rushing_yards', 
       'rushing_tds', 'rushing_fumbles',
       'rushing_fumbles_lost', 'rushing_first_downs', 
       'rushing_2pt_conversions', 

       'targets', 'receptions', 
       'receiving_yards', 'receiving_tds', 
       'receiving_fumbles', 'receiving_fumbles_lost',
       'receiving_first_downs', 'receiving_2pt_conversions'

]

# weekly to seasonal aggregation
season_df = weekly_df[season_cols].groupby(['player_id', 'season', 'player_name', 'position', 'position_group', 'recent_team_old']).agg('sum').reset_index()

# creating a next_season_point
# season_df['next_season_point'] = season_df['season'].shift(-1)

# creating a correlation matrix
# correlation_matrix = season_df.corr()
# season_df.columns
season_df.head()


Index(['player_id', 'season', 'week', 'player_name', 'player_display_name',
       'position', 'position_group', 'recent_team_old', 'opponent_team',
       'season_type', 'completions', 'attempts', 'passing_yards',
       'passing_tds', 'interceptions', 'sacks', 'sack_yards', 'sack_fumbles',
       'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch',
       'passing_first_downs', 'passing_epa', 'passing_2pt_conversions',
       'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles',
       'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa',
       'rushing_2pt_conversions', 'targets', 'receptions', 'receiving_yards',
       'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost',
       'receiving_air_yards', 'receiving_yards_after_catch',
       'receiving_first_downs', 'receiving_epa', 'receiving_2pt_conversions',
       'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr',
       'target_share', 'air_yards_share', 'wopr', 'created_at',